In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import bql 

In [19]:
bq = bql.Service()

In [20]:
import layout_setup
from ipywidgets import Dropdown, Button, VBox, HBox, HTML, Accordion, IntText, FloatText, Label, Textarea, Tab
from ui_helper import CriteriaWidgets, COLS_MAPPING, generate_table, generate_earnings_graph, generate_return_graph,generate_rec_graph, generate_factors_distribution, get_scatter
from criteria_config import criteria_config
from fields_mapping import get_fields, get_grouping_fields, get_additional_fields
from functools import reduce
from function_helper import get_operator_functions, apply_scoring_function
import factors
import ipydatagrid as ipdg
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
# import bqviz as bqv
# import bqplot as bqp
# import bqport

In [21]:
_loading = '<i class="fa fa-spinner fa-spin fa-2x fa-fw" style="color:ivory;"></i>'

In [22]:
def get_user_input():
    return {
        'univ_type': base_univ_choice.value,
        'index_list': index_list.value.split('\n'),
        'tickers_list': tickers_list.value.split('\n'),
        'exchanges': exchange_code_list.value.split(','),
        'criteria': criteria,
        'scoring_function': scoring_function.value,
        'scoring_sector': sector_neutral_function.value
    }

In [23]:
def get_universe(user_input):
    '''
    Returns a bql universe based on user input
    '''
    if user_input['univ_type'] == 'All Equities':
        base_univ = bq.univ.equitiesuniv(['Active', 'Primary']).filter(bq.data.exch_code().in_(user_input['exchanges']))
    elif user_input['univ_type'] == 'Index List':
        base_univ = bq.univ.members(user_input['index_list'])
    elif user_input['univ_type'] == 'Watchlist':
        base_univ = bq.univ.list(user_input['tickers_list'])
    # elif user_input['univ_type'] == 'Portfolio':
    #     base_univ = bq.univ.members(user_input['portfolio'], type='PORT')
    
    fields = get_fields(bq)
    func_operator = get_operator_functions(bq)
    list_criteria = [
        func_operator[crit.get_operator()](fields[crit.get_label()], crit.get_value())
        for crit in user_input['criteria']
    ]
    
    return base_univ.filter(reduce(bq.func.and_, list_criteria))

In [24]:
def get_scoring_fields(user_input):
    grouping_flds = get_grouping_fields(bq)
    grouping = [] if user_input['scoring_sector']=='None' else [grouping_flds[user_input['scoring_sector']]]
    
    factors_fld = {
        'Size': factors.size_mcap(bq),
        'Value': factors.value_B2M(bq),
        'Momentum': factors.mom_12M_minus_1M(bq),
        'Volatility': factors.vol_2y_stdev(bq),
        'Quality': factors.quality_OP2BE(bq)
    }
    
    scores = {
        '{} Score'.format(k): apply_scoring_function(bq, v, user_input['scoring_function'], grouping)
        for k, v in factors_fld.items()
    }
    
    scoring_fields = dict(**factors_fld, **scores)
    scoring_fields['Composite Score'] = bq.func.avg(*[v for v in scores.values()])
    
    return scoring_fields

In [25]:
def get_drilldown(event):
    if event is not None:
        drilldown_status.value = _loading
        drilldown.children = []
        grid = event['owner']
        df_visible = grid.get_visible_data()
        selected_index = event['new'][0]['r1']
        selected_ticker = df_visible.iloc[selected_index][('Description', 'Ticker')]
        selected_industry = df_visible.iloc[selected_index][('Description','BICS Industry')]
        
        peers = df_visible[df_visible[('Description','BICS Industry')]==selected_industry][('Description', 'Ticker')].to_list()
        
        returns = bq.data.day_to_day_total_return(dates=bq.func.range('-6M', '0D')).znav()
        eps = bq.data.is_eps(fpt='A', fpo=bq.func.range('-8','2'))

        dd_flds = {
            'EPS': eps.with_additional_parameters(ae='A'),
            'EPS Est.': eps.with_additional_parameters(ae='E'),
            'Earning Surprise': (eps.with_additional_parameters(ae='A',fs='LR')-eps.with_additional_parameters(ae='E'))/ \
            bq.func.abs(eps.with_additional_parameters(ae='E')),
            '6M Cumul. Return': (1+returns).cumprod()-1,
            '6M Cumul. Return (Industry Avg)': bq.func.value((1+returns.group(returns['date']).avg()).group().cumprod()-1,bq.univ.list(peers)),
            'Target Price': bq.data.best_target_price(dates=bq.func.range('-1Y', '0D'), fill='prev'),
            'Buy Rec': bq.data.tot_buy_rec(dates=bq.func.range('-1Y', '0D'), fill='prev'),
            'Hold Rec': bq.data.tot_hold_rec(dates=bq.func.range('-1Y', '0D'), fill='prev'),
            'Sell Rec': bq.data.tot_sell_rec(dates=bq.func.range('-1Y', '0D'), fill='prev'),
        }

        req = bql.Request(selected_ticker, dd_flds)
        response = bq.execute(req)
        
        df_return = pd.concat([
            response.get('6M Cumul. Return').df().set_index('DATE')[['6M Cumul. Return']], 
            response.get('6M Cumul. Return (Industry Avg)').df().set_index('DATE')[['6M Cumul. Return (Industry Avg)']]], 
            axis=1
        )
        df_rec = pd.concat([
            response.get('Target Price').df().set_index('DATE')[['Target Price']], 
            response.get('Buy Rec').df().set_index('DATE')[['Buy Rec']], 
            response.get('Hold Rec').df().set_index('DATE')[['Hold Rec']], 
            response.get('Sell Rec').df().set_index('DATE')[['Sell Rec']]],
            axis=1
        )
        df_eps = pd.concat([
            response.get('EPS').df().set_index('PERIOD_END_DATE')[['EPS']], 
            response.get('EPS Est.').df().set_index('PERIOD_END_DATE')[['EPS Est.']],],
            axis=1
        )
        
        eps_fig = generate_earnings_graph(df_eps, selected_ticker)
        px_fig = generate_return_graph(df_return, selected_ticker, selected_industry)
        rec_fig = generate_rec_graph(df_rec, selected_ticker)
        
        drilldown.children = [ eps_fig, px_fig, rec_fig]
        drilldown_status.value = drilldown_disclaimer

In [26]:
_ID_COL = ('Description','Ticker')
def update_html(evt=None):
    if evt is not None:
        lst_dict_selection = [dict_selection for dict_selection in evt['new']]
        lst_selection_start_end = [(dict_selection['r1'],dict_selection['r2']+1) 
                                   for dict_selection in lst_dict_selection]
        try:
            df_visible = evt['owner'].get_visible_data()
            if _ID_COL is None:
                lst_index = df_visible.index.to_list()
            else:
            #lst_index = df_visible[(self.id_col,'')].to_list()   #this is for 2 layer indexing, will refactor to support both
                lst_index = df_visible[_ID_COL].to_list()
            #print(lst_index)
            lst_lst_ID = [lst_index[start:end] for (start,end) in lst_selection_start_end]
            #print(lst_lst_ID)
            lst_ID = reduce(lambda a,b: a+b,lst_lst_ID)
            str_ID = ('&#10;').join(lst_ID)
            html_value = '<textarea readonly onClick="this.select()";>'+ str_ID + '</textarea>' #'<input onClick="this.select();" value="'+ str_ID +'"/>'
            html_ticker.value = html_value
            lbl_dnd.value = 'Selected: '
        except:
            pass

In [27]:
def refresh(*args):
    global renamed_df
    results.children = []
    analysis.children = []
    drilldown.children = []
    status.value = _loading
    try:
        user_inputs = get_user_input()
        screen_universe = get_universe(user_inputs)

        sector_flds = get_grouping_fields(bq)
        additional_flds = get_additional_fields(bq)
        scoring_flds = get_scoring_fields(user_inputs)

        request = bql.Request(screen_universe, dict(**sector_flds
                                                    , **additional_flds, **scoring_flds), with_params={'mode':'cached'})
        response = bq.execute(request)

        df = pd.concat([res.df()[res.name] for res in response], axis=1)

        renamed_df = df.rename(columns=COLS_MAPPING)[COLS_MAPPING.values()]

        datagrid_results = generate_table(renamed_df[COLS_MAPPING.values()])
        results_title = HTML('<h2>Screener results ({} names)'.format(len(df)))
        def update_header(evt):
            results_title.value = '<h2>Screener results ({} names)</h2>'.format(len(evt['owner'].get_visible_data()))
        datagrid_results.observe(update_header, '_visible_rows')
        datagrid_results.observe(get_drilldown, 'selections')
        datagrid_results.observe(update_html, 'selections')
        
        drilldown_status.value = drilldown_disclaimer
        results.children = [results_title, drilldown_status, HBox([lbl_dnd, html_ticker]), datagrid_results, drilldown]
        app.selected_index = 1

        factor_dist = generate_factors_distribution(renamed_df)

        score_list = [ fld for cat, fld in renamed_df.columns if cat=='Scores']
        scatter_controls = HBox()
        scatter_box = VBox()
        # controls for the Scatter
        x_dropdown = Dropdown(options=score_list, value=score_list[0], description='X-Axis')
        y_dropdown = Dropdown(options=score_list, value=score_list[1], description='Y-Axis')
        cat_dropdown = Dropdown(options=['GICS Sector', 'BICS Sector', 'BICS Industry','Country'], description='Category')
        scatter_controls.children = [cat_dropdown, x_dropdown, y_dropdown]
        scatter_box.children = [
            get_scatter(
                renamed_df, 
                cat_dropdown.value, 
                x_dropdown.value, 
                y_dropdown.value
            )
        ]

        def on_chg(evt):
            if evt is not None:
                scatter_box.children = [
                    get_scatter(
                        renamed_df, 
                        cat_dropdown.value, 
                        x_dropdown.value, 
                        y_dropdown.value
                    )
                ]
        cat_dropdown.observe(on_chg, 'value')
        x_dropdown.observe(on_chg, 'value')
        y_dropdown.observe(on_chg, 'value')

        analysis.children = [factor_dist, scatter_controls, scatter_box]
        status.value = ''
    except Exception as e:
        status.value = str(e)

In [28]:
default_tickers = ['HLBK MK Equity','AIB MK Equity','BAB MK Equity','DOGT MK Equity','ASTRO MK Equity','QLG MK Equity','PENT MK Equity','SREIT MK Equity','PETD MK Equity','RHBBANK MK Equity','PTG MK Equity','YTLP MK Equity','DIGI MK Equity','KRI MK Equity','MCH MK Equity','SCI MK Equity','MLK MK Equity','UNI MK Equity','FRCB MK Equity','UEMS MK Equity','AXRB MK Equity','UWC MK Equity','ACSM MK Equity','TNB MK Equity','MAG MK Equity','SWB MK Equity','AAGB MK Equity','KPJ MK Equity','TTNP MK Equity','MYEG MK Equity','MMC MK Equity','AXIATA MK Equity','SUCB MK Equity','TDC MK Equity','INRI MK Equity','GDX MK Equity','IHH MK Equity','SDPR MK Equity','SIME MK Equity','SDPL MK Equity','LHIB MK Equity','ABMB MK Equity','SAPE MK Equity','GENP MK Equity','VSI MK Equity','CAB MK Equity','CMS MK Equity','PCHEM MK Equity','BAUTO MK Equity','HART MK Equity','SDH MK Equity','FGV MK Equity','HEIM MK Equity','YTL MK Equity','GREATEC MK Equity','TOPG MK Equity','MBS MK Equity','WPRTS MK Equity','PMAH MK Equity','UMWH MK Equity','PAD MK Equity','IGBREIT MK Equity','HAP MK Equity','YNS MK Equity','BST MK Equity','MRDIY MK Equity','AMM MK Equity','CIMB MK Equity','VITRO MK Equity','IJM MK Equity','GAM MK Equity','MAXIS MK Equity','GENT MK Equity','T MK Equity','BURSA MK Equity','HLFG MK Equity','MFCB MK Equity','MAHB MK Equity','HLI MK Equity','IOI MK Equity','KLK MK Equity','MAY MK Equity','DBB MK Equity','MISC MK Equity','MRC MK Equity','NESZ MK Equity','YTLREIT MK Equity','SKP MK Equity','PEP MK Equity','DRB MK Equity','PBK MK Equity','MI MK Equity','GENM MK Equity','MPI MK Equity','ROTH MK Equity','SPSB MK Equity','GUAN MK Equity','STMB MK Equity','DLG MK Equity','FNH MK Equity']

In [29]:
# Universe UI
# Dropdown to select from all equities or input list of index
universe_label = HTML('<h2>Universe Definition</h2>')
base_univ_choice_label = Label('Base Univ type', layout=layout_setup._label_layout)
base_univ_choice = Dropdown(options=['All Equities', 'Index List', 'Watchlist', 'Portfolio'])
index_list_label = Label('Index List', layout=layout_setup._label_layout)
index_list = Textarea(rows=4, value='MXASJ Index', placeholder='Input your list of indices separated by a new line')
tickers_list_label = Label('Tickers List', layout=layout_setup._label_layout)
tickers_list = Textarea(rows=4, value='\n'.join(default_tickers), placeholder='Input your list of equity tickers separated by a new line')
# port_list = dict(sorted({ p['name']: p['id'] for p in bqport.list_portfolios()}.items()))
# portfolio_label = Label('Portfolio Name', layout=layout_setup._label_layout)
# portfolio_list = Dropdown(options=port_list)

exchange_code_label = Label('Exchange Codes', layout=layout_setup._label_layout)
exchange_code_list = Textarea(rows=2, placeholder='Comma separated list of exchange codes', value='AU,NZ,SP,IN,MK,TB,ID,TT,KS,HK,PM,CH,JT')
base_univ_comp = HBox([exchange_code_label, exchange_code_list])

def on_chg_univ(event):
    if event is not None:
        if event['new'] == 'Index List':
            base_univ_comp.children = [index_list_label, index_list]
        elif event['new'] == 'Watchlist':
            base_univ_comp.children = [tickers_list_label, tickers_list]
        # elif event['new'] == 'Portfolio':
        #     port_list = dict(sorted({ p['name']: p['id'] for p in bqport.list_portfolios()}.items()))
        #     portfolio_list.options = port_list
        #     base_univ_comp.children = [portfolio_label, portfolio_list]
        else:
            base_univ_comp.children = [exchange_code_label, exchange_code_list]

base_univ_choice.observe(on_chg_univ, 'value')
            
criteria = [
    CriteriaWidgets(crit['label'], crit['sign'], crit['limit'], crit['max'], crit['min'] )
    for crit in criteria_config
]

univ_comp = VBox([
    universe_label,
    HBox([base_univ_choice_label, base_univ_choice]),
    base_univ_comp,
] + criteria, layout=layout_setup._input_comp)


In [30]:
# Scoring UI
scoring_label = HTML('<h2>Scoring Methodology</h2>')
scoring_function_label = Label('Scoring Function', layout=layout_setup._label_layout)
scoring_function = Dropdown(options=['Z-score', 'Percentile'], layout=layout_setup._criteria_layout)
sector_neutral_label = Label('Sector-Neutralised', layout=layout_setup._label_layout)
sector_neutral_function = Dropdown(options=['None', 'GICS Sector', 'BICS Sector', 'BICS Industry'], layout=layout_setup._criteria_layout)
scoring_method = HTML('''
<div style="color:ivory;background-color:DimGray;padding:10px;border-radius: 25px;">
    <h3><span style="font-weight:bold"> Factors </span></h3>
    <ul>
        <li><span style="font-weight:bold"> Size </span>: Current Market Capitalisation </li>
        <li><span style="font-weight:bold"> Value </span>: Book to Market </li>
        <li><span style="font-weight:bold"> Momentum </span>: Total Return (-12M to -1M) </li>
        <li><span style="font-weight:bold"> Volatility </span>: 2Y Volatility (Weekly Returns) </li>
        <li><span style="font-weight:bold"> Quality </span>: Operational Income/Common Equity </li>
    </ul>
</div>
''')

scoring_comp = VBox([
    scoring_label,
    HBox([scoring_function_label, scoring_function]),
    HBox([sector_neutral_label, sector_neutral_function]),
    scoring_method
], layout=layout_setup._input_comp)

In [31]:
banner = HTML('<h1>Equity Factor Scoring</h1>')
run_button = Button(description='Run', button_style='info')
run_button.on_click(refresh)
status = HTML()
drilldown_status = HTML()
controls = VBox([
    HBox([univ_comp, scoring_comp]),
    HBox([run_button, status])
])

drilldown_disclaimer = 'Click on a row to drilldown onto the selected security'
lbl_dnd = Label()
html_ticker = HTML(layout={'height': '20px'})


results = VBox(layout={'width':'100%', 'flex-direction':'column'})
drilldown = VBox(layout={'width':'100%', 'flex-direction':'column'}) 
analysis = VBox(layout={'width':'100%', 'flex-direction':'column'})

app = Tab([controls, results, analysis])
app.set_title(0, 'Inputs')
app.set_title(1, 'Results')
app.set_title(2, 'Factors Analysis')

In [32]:
VBox([banner, app])

# 